# Baseline setup

## Load Data

## Resnet or some other model that might be a better base for hierarchical classification

### One classification head first
Start with one classification head that tries to predict all 3(4?) classes at once

### N classification heads

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import time
import copy
import numpy as np



In [23]:
# Loadin saved model
#model_path = "models/your_model_name_model.pth"  # replace with actual name

# Load the full model
#model = torch.load(model_path)

# Set model to evaluation mode (important for inference)
#model.eval()

In [24]:
# Configuration
# The Stanford Cars dataset has 196 classes
NUM_CLASSES = 196
BATCH_SIZE = 32
NUM_EPOCHS = 15
LEARNING_RATE = 0.0001

# Standard ImageNet normalization parameters for ResNet
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]
IMAGE_SIZE = 224 # Standard input size for ResNet

# IMPORTANT: Set your dataset root path here!
DATA_ROOT = './data'

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available and device is set")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS is available and device is set")
else:
    device = torch.device("cpu")
    print("No device is available device set to cpu")

print(f"Using device: {device}")

MPS is available and device is set
Using device: mps


In [25]:
# --- DATA PREPROCESSING ---

# Data Augmentation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),        # Resize slightly larger
        transforms.RandomCrop(224),           # Randomly crop the 224x224 input
        transforms.RandomHorizontalFlip(p=0.5), # Mirror image 50% of time
        transforms.RandomRotation(degrees=15),  # Rotate +/- 15 degrees
        transforms.ColorJitter(brightness=0.1, contrast=0.1), # Slight lighting changes
        transforms.ToTensor(),
        transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
    ]),
    'val': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224),           # Deterministic crop for validation
        transforms.ToTensor(),
        transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
    ]),
}
# Note: You need to update your dataset creation lines to use data_transforms['train'] 
# and data_transforms['val'] appropriately!

try:
    # Load the full training dataset
    full_dataset_for_train = datasets.StanfordCars(root=DATA_ROOT, download=False, split="train", transform=data_transforms['train'])
    full_dataset_for_val = datasets.StanfordCars(root=DATA_ROOT, download=False, split="train", transform=data_transforms['val'])

    test_dataset = datasets.StanfordCars(root=DATA_ROOT, download=False, split="test", transform=data_transforms['val'])
    #full_train_dataset = datasets.ImageFolder(os.path.join(DATA_ROOT, 'cars_train'), data_transforms)
    #test_dataset = datasets.ImageFolder(os.path.join(DATA_ROOT, 'cars_test'), data_transforms)
    
    # Split the original training data into training and validation sets (e.g., 80/20)
    
    # This ensures we pick the SAME images for train/val, but from the correct "dataset version"
    num_train = len(full_dataset_for_train)
    indices = torch.randperm(num_train, generator=torch.Generator().manual_seed(42)).tolist()
    
    split_point = int(0.8 * num_train)
    train_idx = indices[:split_point]
    val_idx = indices[split_point:]

    # train_dataset pulls from the Augmented loader
    # val_dataset pulls from the Clean loader 
    train_dataset = torch.utils.data.Subset(full_dataset_for_train, train_idx)
    val_dataset   = torch.utils.data.Subset(full_dataset_for_val, val_idx)

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    dataloaders = {'train': train_loader, 'val': val_loader}
    dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

    class_names = full_dataset_for_val.classes
    idx_to_class = {i: class_name for i, class_name in enumerate(class_names)}

except FileNotFoundError as e:
    print(f"\n[ERROR] Data not found. Please check your DATA_ROOT path: {DATA_ROOT}")
    print("Ensure you have 'train' and 'test' subdirectories with images inside.")

In [26]:
# Extract the make (coarse class) from full class name
def get_make(class_name):
    if class_name.startswith("Aston Martin"): return "Aston Martin"
    if class_name.startswith("Land Rover"): return "Land Rover"
    if class_name.startswith("AM General"): return "AM General"
    return class_name.split()[0]

In [27]:
# --- Generate Class-to-Make Mappings ---

# Get list of all class names
all_classes = full_dataset_for_val.classes

# 1. extract unique makes
unique_makes = sorted(list(set([get_make(c) for c in all_classes])))
num_makes = len(unique_makes)

# 2. Create dictionaries
make_to_idx = {make_name: i for i, make_name in enumerate(unique_makes)}
idx_to_make = {i: make_name for i, make_name in enumerate(unique_makes)}

# 3. Create a mapping tensor for the training loop
# input: class_index (0-195) -> output: make_index (0-48)
class_to_make_tensor = torch.zeros(len(all_classes), dtype=torch.long)

for class_idx, class_name in enumerate(all_classes):
    make_name = get_make(class_name)
    make_idx = make_to_idx[make_name]
    class_to_make_tensor[class_idx] = make_idx

print(f"Found {num_makes} unique makes.")
print(f"Mapping prepared. Example: Class 0 ({all_classes[0]}) -> Make ID {class_to_make_tensor[0]} ({unique_makes[class_to_make_tensor[0]]})")

Found 49 unique makes.
Mapping prepared. Example: Class 0 (AM General Hummer SUV 2000) -> Make ID 0 (AM General)


In [28]:
class MultiHeadResNet(nn.Module):
    def __init__(self, num_classes, num_makes):
        super(MultiHeadResNet, self).__init__()
        
        # 1. Load the backbone (ResNet50)
        backbone = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        
        # 2. Remove the existing final layer (fc)
        # We want everything *before* the classification layer
        self.features = nn.Sequential(*list(backbone.children())[:-1])
        
        # 3. Create the two heads
        # ResNet50 output size is 2048
        self.fc_make = nn.Linear(2048, num_makes)   # Head 1: Predicts Brand
        self.fc_model = nn.Linear(2048, num_classes) # Head 2: Predicts Car Model
        
    def forward(self, x):
        # Pass through the frozen/unfrozen feature extractor
        x = self.features(x)
        x = torch.flatten(x, 1) # Flatten for linear layers
        
        # Branch into two outputs
        out_make = self.fc_make(x)
        out_model = self.fc_model(x)
        
        return out_make, out_model

# Initialize the model
model_multihead = MultiHeadResNet(NUM_CLASSES, num_makes)
if device == torch.device("mps"):
    model_multihead.float()
model_multihead = model_multihead.to(device)

# Criterion
criterion = nn.CrossEntropyLoss()

# Define Optimizer (Standard fine-tuning LR)
optimizer = optim.Adam(model_multihead.parameters(), lr=0.0001)

In [29]:
# --- MODEL SETUP (ResNet-50 Feature Extractor) ---

def setup_resnet_baseline(num_classes):
    # 1. Load Pretrained ResNet-50
    # Use the default ImageNet weights
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

    # 2. Freeze all convolutional layers (Feature Extraction)
    # This is the "Transfer Learning" step. Only the final classification head will learn.
    for param in model.parameters():
        param.requires_grad = True

    # 3. Replace the final Fully Connected layer (the "Head")
    # Get the input feature size of the original FC layer (2048 for ResNet-50)
    num_ftrs = model.fc.in_features

    # Map the features to the new number of classes (196)
    model.fc = nn.Linear(num_ftrs, num_classes)
    
    # NOTE: Only model.fc's parameters are now set to requires_grad=True
    return model

#model_baseline = setup_resnet_baseline(NUM_CLASSES)
#if device == torch.device("mps"):
 #   model_baseline.float()
#model_baseline.to(device)

# Define Loss function and Optimizer
#criterion = nn.CrossEntropyLoss()

# Only pass the parameters of the un-frozen layers (the new 'fc' layer)
#optimizer = optim.Adam(model_baseline.parameters(), lr=LEARNING_RATE)

In [30]:
from experiment_logging import ExperimentLogger

logger = ExperimentLogger(
    log_file="experiment_log.md",
    active=True,         # set False to disable logging
    show_console=True    # preview logs in notebook
)

# Base info for this experiment
logger.set(
    name="Multihead_two_heads_with_data_augmentation"  ,
    changes="Added data augmentation",
    reason="Try to prevent the model from overfitting, and force the network to learn more robust features that remain consistent regardless of position or lighting"
)

In [31]:
def accuracy_topk(outputs, targets, topk=(1,)):
    """Compute the top-k accuracies."""
    maxk = max(topk)
    batch_size = targets.size(0)

    _, pred = outputs.topk(maxk, dim=1, largest=True, sorted=True)
    pred = pred.t()
    correct = pred.eq(targets.view(1, -1).expand_as(pred))

    results = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        acc = correct_k * (100.0 / batch_size)
        results.append(acc.item())
    return results

In [32]:
def train_model(model, criterion, optimizer, num_epochs=NUM_EPOCHS):

    since = time.time()

    # Move the mapping tensor to the same device as the model (GPU/CPU)
    class_to_make_tensor_gpu = class_to_make_tensor.to(device)

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc= 0.0 # best top-1 accuracy

    # Store epoch metrics
    train_losses, val_losses = [], []
    train_top1s, val_top1s = [], []
    train_top5s, val_top5s = [], []

    print("Starting Training...")

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 20)

        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_top1 = 0.0
            running_top5 = 0.0

            correct_makes = 0
            correct_models = 0
            total_samples = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device) # These are specific model labels (0-195)

                # Use the global mapping tensor to generate make labels on the fly
                make_labels = class_to_make_tensor_gpu[labels]

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    out_make, out_model = model(inputs)

                    # Calculate the loss for BOTH heads
                    loss_make = criterion(out_make, make_labels)
                    loss_model = criterion(out_model, labels)

                    # Combined loss (we can weigh these differently)
                    total_loss = 1 * loss_make + 1 * loss_model

                    # ---- TOP-1 / TOP-5 ----
                    top1, top5 = accuracy_topk(out_model, labels, topk=(1, 5))

                    # Backprop only in training
                    if phase == "train":
                        total_loss.backward()
                        optimizer.step()

                batch_size = inputs.size(0)
                running_loss += total_loss.item() * batch_size
                running_top1 += top1 * batch_size
                running_top5 += top5 * batch_size

                # Accuracy for Make head
                _, preds_make = torch.max(out_make, 1)
                correct_makes += torch.sum(preds_make == make_labels.data)

                # Accuracy for Model head
                _, preds_model = torch.max(out_model, 1)
                correct_models += torch.sum(preds_model == labels.data)
                
                total_samples += batch_size

            # Compute epoch metrics
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_top1 = running_top1 / dataset_sizes[phase]
            epoch_top5 = running_top5 / dataset_sizes[phase]


            if device != torch.device("mps"):
                epoch_make_acc = correct_makes.double() / dataset_sizes[phase]
                epoch_model_acc = correct_models.double() / dataset_sizes[phase]
            else:
                epoch_make_acc = correct_makes.float() / dataset_sizes[phase]
                epoch_model_acc = correct_models.float() / dataset_sizes[phase] 
            
            epoch_top1_val = epoch_top1.item() if isinstance(epoch_top1, torch.Tensor) else float(epoch_top1)
            epoch_top5_val = epoch_top5.item() if isinstance(epoch_top5, torch.Tensor) else float(epoch_top5)

            print(f"{phase:5} Loss: {epoch_loss:.4f}  Top-1: {epoch_top1_val:.2f}%  Top-5: {epoch_top5_val:.2f}%")

            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} | Make Acc: {epoch_make_acc:.2%} | Model Acc: {epoch_model_acc:.2%}")

            # Store metrics for plotting/logging
            if phase == "train":
                train_losses.append(epoch_loss)
                train_top1s.append(epoch_top1_val)
                train_top5s.append(epoch_top5_val)
            else:
                val_losses.append(epoch_loss)
                val_top1s.append(epoch_top1_val)
                val_top5s.append(epoch_top5_val)
            
                # Log to experiment logger
                logger.record_metrics(
                    epoch=epoch + 1,
                    train_loss=train_losses[-1],
                    train_top1=train_top1s[-1],
                    train_top5=train_top5s[-1],
                    val_loss=val_losses[-1],
                    val_top1=val_top1s[-1],
                    val_top5=val_top5s[-1]
                )

                # Update best model
                if epoch_top1_val > best_acc:
                    best_acc = epoch_top1_val
                    best_model_wts = copy.deepcopy(model.state_dict())

    # ------------------------------------------------------
    # END OF TRAINING LOOP
    # ------------------------------------------------------

    time_elapsed = time.time() - since
    print(f"\nTraining complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s")
    print(f"Best validation Top-1 Acc: {best_acc:.2f}%")


    model.load_state_dict(best_model_wts)

    # ------------------------------------------------------
    # Loss curve plot
    # ------------------------------------------------------
    fig, ax = plt.subplots()
    ax.plot(train_losses, label="Train Loss")
    ax.plot(val_losses, label="Val Loss")
    ax.set_title("Loss Curve")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.legend()
    logger.add_plot(fig, name="loss_curve")

    # ------------------------------------------------------
    # Accuracy curve plot
    # ------------------------------------------------------
    fig2, ax2 = plt.subplots()
    ax2.plot(train_top1s, label="Train Top-1")
    ax2.plot(val_top1s, label="Val Top-1")
    ax2.plot(train_top5s, label="Train Top-5")
    ax2.plot(val_top5s, label="Val Top-5")
    ax2.set_title("Accuracy Curve")
    ax2.set_xlabel("Epoch")
    ax2.set_ylabel("Accuracy (%)")
    ax2.legend()
    logger.add_plot(fig2, name="accuracy_curve")

    # ------------------------------------------------------
    # Confusion matrix
    # ------------------------------------------------------
    all_preds = []
    val_labels = [label for _, label in val_dataset]

    model.eval()
    with torch.no_grad():
        for inputs, _ in dataloaders["val"]:
            inputs = inputs.to(device)
            out_make, out_model = model(inputs)
            _, preds = torch.max(out_model, 1)
            all_preds.extend(preds.cpu().numpy())
    
    # ------------------------------------------------------
    # Compute hierarchical metrics
    # ------------------------------------------------------
    idx_to_class = {i: name for i, name in enumerate(full_dataset_for_val.classes)}
    pred_class_names = [idx_to_class[p] for p in all_preds]
    true_class_names = [idx_to_class[t] for t in val_labels]

    pred_makes = [get_make(c) for c in pred_class_names]
    true_makes = [get_make(c) for c in true_class_names]

    # Hierarchical consistency
    hier_consistency = np.mean([p == t for p, t in zip(pred_makes, true_makes)])
    print(f"Hierarchical Consistency: {hier_consistency:.4f}")

    # Per-make accuracy
    from collections import defaultdict
    make_correct = defaultdict(int)
    make_total = defaultdict(int)
    for p_make, t_make in zip(pred_makes, true_makes):
        make_total[t_make] += 1
        if p_make == t_make:
            make_correct[t_make] += 1
    per_make_acc = {make: make_correct[make] / make_total[make] for make in make_total}
    print("Per-make Accuracy:", per_make_acc)

    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(val_labels, all_preds)
    logger.add_confusion_matrix(cm, class_names=full_dataset_for_val.classes)
    logger.add_scalar("hierarchical_consistency", hier_consistency)
    for make, acc in per_make_acc.items():
        logger.add_scalar(f"accuracy_{make}", acc)


    # ------------------------------------------------------
    # Save sample validation image
    # ------------------------------------------------------
    sample_img, _ = next(iter(dataloaders["val"]))
    logger.add_image(sample_img[0], name="sample_val_image")

    # ------------------------------------------------------
    # Write final log summary
    # ------------------------------------------------------
    logger.results = f"Best Top-1 Accuracy = {best_acc:.2f}%"
    logger.results += f", Hierarchical Consistency = {hier_consistency:.4f}"
    logger.notes = "Training finished. Logged curves, confusion matrix, sample image."
    logger.commit()

    # ------------------------------------------------------
    # Save trained model
    # ------------------------------------------------------
    model_path = f"models/{logger.name}_model.pth"
    os.makedirs("models", exist_ok=True)
    torch.save(model, model_path)
    print(f"Model saved to {model_path}")

    return model


In [33]:
def check_setup():
    print("\n--- Running Setup Health Check ---")
    
    # 1. Device check (already done, but reconfirm)
    print(f"Model is on: {next(model_multihead.parameters()).device}")
    
    # 2. Data Loader Check (load a single batch)
    try:
        inputs, labels = next(iter(train_loader))
        
        # 3. Check for float32 enforcement and device transfer
        inputs = inputs.to(device, dtype=torch.float32)
        labels = labels.to(device)
        
        # Check input tensor properties
        print(f"Input batch shape: {inputs.shape}")
        print(f"Input tensor dtype: {inputs.dtype}")
        print(f"Input tensor device: {inputs.device}")
        
    except Exception as e:
        print(f"[CRITICAL ERROR] Failed to load or transfer batch. Fix this before training.")
        print(f"Error details: {e}")
        return

    # 4. Model Forward Pass Check
    try:
        model_multihead.eval() # Set to eval mode for the check
        with torch.no_grad():
            out_make ,out_model = model_multihead(inputs)
            
        print(f"out_make shape (Batch x Classes): {out_make.shape}")
        print(f"out_model shape (Batch x Classes): {out_model.shape}")
        print("Setup check PASSED! Ready for full training.")
        
    except Exception as e:
        print(f"[CRITICAL ERROR] Failed during forward pass.")
        print(f"Error details: {e}")

# check_setup() # Run this once before calling train_model

In [34]:

# To run the training:
check_setup()
final_baseline_model = train_model(model_multihead, criterion, optimizer, num_epochs=NUM_EPOCHS)


--- Running Setup Health Check ---
Model is on: mps:0
Input batch shape: torch.Size([32, 3, 224, 224])
Input tensor dtype: torch.float32
Input tensor device: mps:0
out_make shape (Batch x Classes): torch.Size([32, 49])
out_model shape (Batch x Classes): torch.Size([32, 196])
Setup check PASSED! Ready for full training.
Starting Training...

Epoch 1/15
--------------------
train Loss: 7.8159  Top-1: 6.35%  Top-5: 18.63%
Train Loss: 7.8159 | Make Acc: 22.96% | Model Acc: 6.35%
val   Loss: 5.8696  Top-1: 17.13%  Top-5: 49.23%
Val Loss: 5.8696 | Make Acc: 45.79% | Model Acc: 17.13%

Epoch 2/15
--------------------
train Loss: 4.4404  Top-1: 37.21%  Top-5: 71.27%
Train Loss: 4.4404 | Make Acc: 58.04% | Model Acc: 37.21%
val   Loss: 2.9865  Top-1: 53.78%  Top-5: 86.13%
Val Loss: 2.9865 | Make Acc: 73.17% | Model Acc: 53.78%

Epoch 3/15
--------------------
train Loss: 2.2661  Top-1: 65.74%  Top-5: 91.99%
Train Loss: 2.2661 | Make Acc: 78.86% | Model Acc: 65.74%
val   Loss: 1.8759  Top-1: 69

In [38]:
def evaluate_model(model, loader, dataset_size, device, idx_to_class):
    print("\n--- Starting Phase 1 Baseline Evaluation ---")
    
    model.eval()
    correct_exact = 0
    correct_make = 0
    total_samples = 0

    # Create a fast lookup table for Make names to speed up the loop
    # idx -> "BMW", "Ford", etc.
    idx_to_make = {i: get_make(name) for i, name in idx_to_class.items()}

    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device, dtype=torch.float32)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            # Move to CPU for list comparison (faster than GPU for string logic)
            preds_np = preds.cpu().numpy()
            labels_np = labels.cpu().numpy()

            for i in range(len(labels_np)):
                pred_idx = preds_np[i]
                true_idx = labels_np[i]

                # 1. Exact Match Check (Standard Top-1)
                if pred_idx == true_idx:
                    correct_exact += 1
                
                # 2. Make Match Check (Hierarchical)
                pred_make = idx_to_make[pred_idx]
                true_make = idx_to_make[true_idx]
                
                if pred_make == true_make:
                    correct_make += 1
                
                total_samples += 1

    # Calculate percentages
    exact_acc = correct_exact / total_samples
    make_acc = correct_make / total_samples
    gap = make_acc - exact_acc

    print(f"Test Set Size: {total_samples}")
    print("-" * 40)
    print(f"1. Exact Car Accuracy (Hard):  {exact_acc:.2%}  (Target for fine-tuning)")
    print(f"2. Make/Brand Accuracy (Easy): {make_acc:.2%}  (Target for coarse features)")
    print("-" * 40)
    print(f"3. THE GAP:                    {gap:.2%} points")
    print(f"   (This is the specific error margin your future Multi-Head model will target)")
    
    return exact_acc, make_acc

In [39]:
def evaluate_multihead_model(model, loader, device, class_to_make_tensor):
    print("\n--- Starting Phase 3 (Multi-Head) Evaluation ---")
    
    model.eval()
    
    # Move the lookup tensor to the same device as the model
    class_to_make_tensor = class_to_make_tensor.to(device)
    
    correct_make_head = 0
    correct_model_head = 0
    consistent_predictions = 0
    total_samples = 0
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # 1. Generate the TRUE Make labels for this batch
            true_make_labels = class_to_make_tensor[labels]
            
            # 2. Forward pass (Get TWO outputs)
            out_make, out_model = model(inputs)
            
            # 3. Get Predictions
            _, preds_make = torch.max(out_make, 1)   # Head 1 prediction
            _, preds_model = torch.max(out_model, 1) # Head 2 prediction
            
            # --- METRIC 1: Head Accuracy ---
            correct_make_head += torch.sum(preds_make == true_make_labels).item()
            correct_model_head += torch.sum(preds_model == labels).item()
            
            # --- METRIC 2: Consistency Check ---
            # Does Head 1 (Make) agree with Head 2 (Model)?
            # Logic: Look up the parent Make of the *predicted* Model.
            # If that equals the *predicted* Make, they are consistent.
            parent_of_predicted_model = class_to_make_tensor[preds_model]
            consistent_predictions += torch.sum(preds_make == parent_of_predicted_model).item()
            
            total_samples += inputs.size(0)

    # Calculate Final Percentages
    acc_make = correct_make_head / total_samples
    acc_model = correct_model_head / total_samples
    consistency = consistent_predictions / total_samples
    gap = acc_make - acc_model

    print(f"Test Set Size: {total_samples}")
    print("-" * 40)
    print(f"1. Model Head Accuracy:    {acc_model:.2%} (The Hard Task)")
    print(f"2. Make Head Accuracy:     {acc_make:.2%} (The Easy Task)")
    print("-" * 40)
    print(f"3. THE GAP:                {gap:.2%} points")
    print(f"4. Internal Consistency:   {consistency:.2%} (How often heads agree)")
    
    return acc_model, acc_make, consistency

In [40]:
# 1. Load the best weights saved during training
best_model_test = setup_resnet_baseline(NUM_CLASSES)
best_model_test.to(device).float() # Ensure float32 and device transfer
# best_model_test.load_state_dict(torch.load('best_baseline_weights.pth', map_location=device))

# 2. Run the evaluation
# New version
idx_to_class = {i: c for i, c in enumerate(full_dataset_for_val.classes)}
acc_exact, acc_make, consistency = evaluate_multihead_model(final_baseline_model, test_loader, device, class_to_make_tensor)


--- Starting Phase 3 (Multi-Head) Evaluation ---
Test Set Size: 8041
----------------------------------------
1. Model Head Accuracy:    85.65% (The Hard Task)
2. Make Head Accuracy:     92.71% (The Easy Task)
----------------------------------------
3. THE GAP:                7.06% points
4. Internal Consistency:   95.26% (How often heads agree)


In [ ]:
def evaluate_model(model, loader, dataset_size, device):
    print("\n--- Starting Test Set Evaluation ---")
    
    model.eval() # Set model to evaluation mode
    running_corrects = 0
    
    # Top-5 accuracy tracking
    top5_corrects = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in loader:
            
            # Ensure consistent device and dtype (float32)
            inputs = inputs.to(device, dtype=torch.float32)
            labels = labels.to(device)
            
            outputs = model(inputs)
            
            # --- Top-1 Accuracy ---
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
            
            # --- Top-5 Accuracy (Good for Fine-Grained Tasks) ---
            # Get the top 5 predicted classes
            _, top5_preds = torch.topk(outputs, 5, dim=1)
            
            # Check if the true label is in the top 5 predictions
            labels_reshaped = labels.view(-1, 1) # Shape [Batch, 1]
            top5_corrects += torch.sum(top5_preds.eq(labels_reshaped)).item()
            total_samples += inputs.size(0)

    # Calculate final metrics
    if device != torch.device("mps"):
        top1_acc = running_corrects.double() / dataset_size
    else:
        top1_acc = running_corrects.float() / dataset_size
    top5_acc = top5_corrects / total_samples
    
    print(f'Test Set Size: {dataset_size}')
    print(f'Final Baseline Top-1 Accuracy: {top1_acc.item():.4f}')
    print(f'Final Baseline Top-5 Accuracy: {top5_acc:.4f}')
    
    return top1_acc.item(), top5_acc

# --- Example of running the test ---

# 1. Load the best weights saved during training
best_model_test = setup_resnet_baseline(NUM_CLASSES)
best_model_test.to(device).float() # Ensure float32 and device transfer
# best_model_test.load_state_dict(torch.load('best_baseline_weights.pth', map_location=device))

# 2. Run the evaluation
test_top1_acc, test_top5_acc = evaluate_model(final_baseline_model, test_loader, len(test_dataset), device)


--- Starting Test Set Evaluation ---
Test Set Size: 8041
Final Baseline Top-1 Accuracy: 0.7592
Final Baseline Top-5 Accuracy: 0.9353


In [ ]:
import torch
import numpy as np

def get_predictions_and_labels(model, loader, device):
    """
    Runs the model on the data loader and collects all predictions, 
    true labels, and output logits.
    """
    model.eval()
    all_preds = []
    all_labels = []
    all_outputs = []

    with torch.no_grad():
        for inputs, labels in loader:
            # Ensure consistent device and dtype (float32)
            inputs = inputs.to(device, dtype=torch.float32)
            labels = labels.to(device)

            outputs = model(inputs)
            
            # Get Top-1 prediction index
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_outputs.extend(outputs.cpu().numpy())

    return np.array(all_preds), np.array(all_labels), np.array(all_outputs)

# Example usage (assuming 'best_model_test', 'test_loader', 'device' are defined):
test_preds, test_labels, test_outputs = get_predictions_and_labels(
    final_baseline_model, test_loader, device)


In [ ]:
class_names = full_train_dataset.classes
idx_to_class = {i: class_name for i, class_name in enumerate(class_names)}

In [ ]:
# 1. Find indices where prediction does not match label
incorrect_indices = np.where(test_preds != test_labels)[0]

# 2. Total errors should match (1 - Accuracy)
num_incorrect = len(incorrect_indices)
print(f"Total Incorrect Predictions: {num_incorrect}")
print(f"Total Test Samples: {len(test_labels)}")
print(f"Error Rate (Expected): {1 - 0.8832:.4f} vs Actual: {num_incorrect / len(test_labels):.4f}")

# 3. Print a few specific errors to check for patterns
# Print a few specific errors to check for patterns
for i, idx in enumerate(incorrect_indices[:5]):
    predicted_class_idx = test_preds[idx]
    true_class_idx = test_labels[idx]
    
    # Map the indices to their human-readable names
    predicted_class_name = idx_to_class[predicted_class_idx]
    true_class_name = idx_to_class[true_class_idx]
    
    print(f"\nError #{i+1} (Index {idx}):")
    print(f"  Predicted Class: **{predicted_class_name}**")
    print(f"  True Class:      **{true_class_name}**")

Total Incorrect Predictions: 1936
Total Test Samples: 8041
Error Rate (Expected): 0.1168 vs Actual: 0.2408

Error #1 (Index 7):
  Predicted Class: **Toyota Camry Sedan 2012**
  True Class:      **Mercedes-Benz S-Class Sedan 2012**

Error #2 (Index 18):
  Predicted Class: **Audi TT Hatchback 2011**
  True Class:      **Audi S5 Coupe 2012**

Error #3 (Index 23):
  Predicted Class: **Audi TTS Coupe 2012**
  True Class:      **Ford Mustang Convertible 2007**

Error #4 (Index 27):
  Predicted Class: **Buick Rainier SUV 2007**
  True Class:      **Volvo XC90 SUV 2007**

Error #5 (Index 31):
  Predicted Class: **Dodge Journey SUV 2012**
  True Class:      **Jeep Grand Cherokee SUV 2012**
